In [1]:
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, accuracy_score

# from collections import Counter

In [2]:
data = pd.read_csv('../data/train_events.csv')
video = pd.read_csv('../data/video_info_v2.csv')
targets = pd.read_csv('../data/train_targets.csv')
all_events = pd.read_csv('../data/all_events.csv')

In [4]:
data.head() # тут можно найти какие пользователи были и что смотрели

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976


In [5]:
video.head() # тут инфа о просмотренном видео

,rutube_video_id,title,category,duration,author_id
0,video_185549,Как собрать букет из мыльных тюльпанов - Силик...,Хобби,1559160,1015054
1,video_111035,"Осторожно, Киберземляне!, 1 сезон, 12 серия",Сериалы,1320007,1002180
2,video_476517,ПОПУЛЯРНЫЕ ВИДЕОИГРЫ в LEGO... перевод - TD BR...,Хобби,606145,1095337
3,video_157198,"Хороший лжец (фильм, 2019)",Фильмы,6577440,1043618
4,video_289824,Нашего старого гнобят по-всякому,Развлечения,859493,1009535


In [6]:
video['title'][1]

'Осторожно, Киберземляне!, 1 сезон, 12 серия'

In [123]:
targets.head() # соцдемографическая информация о пользователях

,viewer_uid,age,sex,age_class,region,total_watchtime
0,10087154,30,male,1,Krasnoyarsk Krai,511
1,10087154,30,male,1,Krasnoyarsk Krai,531
2,10087154,30,male,1,Novosibirsk Oblast,511
3,10087154,30,male,1,Novosibirsk Oblast,531
4,10908708,25,female,1,St.-Petersburg,68


In [59]:
all_events.head() 

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 13:08:30+03:00,Tatarstan Republic,smartphone,browser,Android,Chrome Mobile,60,video_395879,10813370
1,2024-06-01 14:30:00+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,60,video_216518,10512324
2,2024-06-01 18:48:12+03:00,Novosibirsk Oblast,smartphone,mobile app,Android,Rutube,121,video_41225,10951137
3,2024-06-01 16:32:36+03:00,Moscow,desktop,browser,Windows,Yandex Browser,2324,video_215886,10912434
4,2024-06-01 00:57:04+03:00,Moscow Oblast,smartphone,mobile app,Android,Rutube,6830,video_43631,10223585


    В таблице data указаны события пользователей Rutube 
    - event_date : Дата события
    - viewer_uid : Идентификатор пользователя
    - region : Регион пользователя
    - rutube_video_id : Идентификатор видео
    - ua_device_type : Устройство пользователя
    - ua_client_type : Приложение/браузер 
    - ua_os : Операционная система устройства пользователя
    - ua_client_name : Веб-браузер/приложение, с которого пользователь просматривал видео
    - total_watchtime : Время просмотра в секундах

    Выделим валидационную выборку, на которой оценим качество нашего решения. Для этого разделим датасет случайным 
    образом по айди пользователей

In [60]:
len(data.viewer_uid.unique())

180012

In [61]:
len(all_events.viewer_uid.unique())

960885

In [3]:
result = pd.concat([data, all_events], axis=0).drop_duplicates()
result = result.reset_index(drop=True)

In [9]:
result.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976


In [64]:
len(result.viewer_uid.unique())

1140897

In [65]:
def create_user_history_sorted(df):
    # Сортируем по времени
    df = df.sort_values('event_timestamp')
    
    # Убираем префикс 'video_' из rutube_video_id
    df['clean_video_id'] = df['rutube_video_id'].str.replace('video_', '')
    
    # Группируем по пользователю и собираем список просмотренных видео
    user_history = df.groupby('viewer_uid')['clean_video_id'].agg(list).reset_index()
    
    # Оставляем только пользователей с более чем одним просмотром
    user_history = user_history[user_history['clean_video_id'].map(len) > 1]
    
    # Сбрасываем индекс
    user_history = user_history.reset_index(drop=True)
    
    return user_history

# Пример использования:
user_history_df = create_user_history_sorted(result)

# Посмотреть результат
print(user_history_df.head())

   viewer_uid                                     clean_video_id
0    10000001  [128965, 298886, 346543, 41327, 96775, 405852,...
1    10000002  [314025, 198798, 166641, 474304, 148064, 27449...
2    10000003  [334752, 363588, 135622, 310140, 56820, 387408...
3    10000004  [481044, 371881, 342531, 316595, 481329, 22340...
4    10000007  [4397, 120267, 340242, 338727, 230330, 278857,...


In [66]:
len(user_history_df.viewer_uid.unique())

907983

In [67]:
len(user_history_df['clean_video_id'][2])

27

In [68]:
# Способ 2: Найти максимальную длину по всем спискам
max_length = max(user_history_df['clean_video_id'].map(len))

In [69]:
max_length

401

In [70]:
# Объединяем датафреймы по колонке 'rutube_video_id'
result_with_video_info = pd.merge(result, video, on='rutube_video_id', how='left')
result_with_video_info.head()

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243,Папа с особенностями. Мужское / Женское. Выпус...,Телепередачи,2456534,1009219
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341,Comedy Club: Мальдивы | Андрей Бебуришвили,Юмор,519211,1006760
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333,"Новая Битва экстрасенсов, 24 сезон, 11 выпуск",Телепередачи,5518280,1009257
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092,Сергей Орлов-снял дом!!!,Разное,1522069,1058671
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976,Ищем сокровища в Полевском | Уральская Флоренц...,Путешествия,1249920,1020020


In [4]:
# Выбираем только колонки viewer_uid и region из result
regions = result[['viewer_uid', 'region']].drop_duplicates()

# Добавляем region к targets
targets = pd.merge(targets, regions, on='viewer_uid', how='left')

# Посмотреть результат
targets.head()

,viewer_uid,age,sex,age_class,region
0,10087154,30,male,1,Krasnoyarsk Krai
1,10087154,30,male,1,Novosibirsk Oblast
2,10908708,25,female,1,St.-Petersburg
3,10908708,25,female,1,Moscow Oblast
4,10190464,34,male,2,Irkutsk Oblast


In [5]:
# Выбираем только колонки viewer_uid и region из result
regions = result[['viewer_uid', 'total_watchtime']].drop_duplicates()

# Добавляем region к targets
targets = pd.merge(targets, regions, on='viewer_uid', how='left')

# Посмотреть результат
targets.head()

,viewer_uid,age,sex,age_class,region,total_watchtime
0,10087154,30,male,1,Krasnoyarsk Krai,511
1,10087154,30,male,1,Krasnoyarsk Krai,531
2,10087154,30,male,1,Novosibirsk Oblast,511
3,10087154,30,male,1,Novosibirsk Oblast,531
4,10908708,25,female,1,St.-Petersburg,68


In [73]:
result_with_video_info = result_with_video_info.drop('region', axis=1)
result_with_video_info.head()

,event_timestamp,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id
0,2024-06-01 06:40:58+03:00,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243,Папа с особенностями. Мужское / Женское. Выпус...,Телепередачи,2456534,1009219
1,2024-06-01 19:33:24+03:00,smartphone,mobile app,Android,Rutube,512,video_362960,10245341,Comedy Club: Мальдивы | Андрей Бебуришвили,Юмор,519211,1006760
2,2024-06-01 21:30:43+03:00,desktop,browser,Windows,Chrome,5647,video_96775,10894333,"Новая Битва экстрасенсов, 24 сезон, 11 выпуск",Телепередачи,5518280,1009257
3,2024-06-01 23:03:42+03:00,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092,Сергей Орлов-снял дом!!!,Разное,1522069,1058671
4,2024-06-01 22:48:09+03:00,smartphone,mobile app,Android,Rutube,71,video_116245,10452976,Ищем сокровища в Полевском | Уральская Флоренц...,Путешествия,1249920,1020020


In [74]:
result_with_video_info = result_with_video_info.drop('total_watchtime', axis=1)
result_with_video_info.head()

,event_timestamp,ua_device_type,ua_client_type,ua_os,ua_client_name,rutube_video_id,viewer_uid,title,category,duration,author_id
0,2024-06-01 06:40:58+03:00,desktop,browser,Windows,Yandex Browser,video_133074,10067243,Папа с особенностями. Мужское / Женское. Выпус...,Телепередачи,2456534,1009219
1,2024-06-01 19:33:24+03:00,smartphone,mobile app,Android,Rutube,video_362960,10245341,Comedy Club: Мальдивы | Андрей Бебуришвили,Юмор,519211,1006760
2,2024-06-01 21:30:43+03:00,desktop,browser,Windows,Chrome,video_96775,10894333,"Новая Битва экстрасенсов, 24 сезон, 11 выпуск",Телепередачи,5518280,1009257
3,2024-06-01 23:03:42+03:00,smartphone,mobile app,Android,Rutube,video_161610,10029092,Сергей Орлов-снял дом!!!,Разное,1522069,1058671
4,2024-06-01 22:48:09+03:00,smartphone,mobile app,Android,Rutube,video_116245,10452976,Ищем сокровища в Полевском | Уральская Флоренц...,Путешествия,1249920,1020020


In [92]:
date_range = result_with_video_info['event_timestamp'].agg(['min', 'max'])
print("Диапазон дат:")
print(f"От: {date_range['min']}")
print(f"До: {date_range['max']}")

Диапазон дат:
От: 2024-06-01 00:00:00+03:00
До: 2024-06-30 23:59:28+03:00


In [117]:
def create_detailed_user_history(df): 
    month_dict = {
        'June': 'июня'
        # добавьте остальные месяцы по необходимости
    }
    
    # Форматируем дату
    df['formatted_date'] = pd.to_datetime(df['event_timestamp']).dt.strftime('%d %B')
    df['formatted_date'] = df['formatted_date'].apply(
        lambda x: f"{x.split()[0]} {month_dict.get(x.split()[1], x.split()[1])}"
    )

    # Определяем тип клиента
    df['client_type'] = df['ua_client_type'].apply(
        lambda x: 'браузере' if x == 'browser' else 'приложении' if x == 'mobile app' else x
    )
    
    # Создаем детальное описание каждого просмотра
    def create_view_description(row):
        parts = []
        
        if pd.notna(row['title']):
            parts.append(str(row['title']))
            
        if pd.notna(row['category']):
            parts.append(str(row['category']))
            
        if pd.notna(row['client_type']):
            parts.append(f"просмотрено в {row['client_type']}")
            
        if pd.notna(row['ua_os']):
            parts.append(f"ОС {row['ua_os']}")
            
        if pd.notna(row['formatted_date']):
            parts.append(str(row['formatted_date']))
            
        return ' '.join(parts) if parts else None
    
    df['detailed_view'] = df.apply(create_view_description, axis=1)
    
    # Группируем по пользователю и собираем список просмотров
    user_history = df.groupby('viewer_uid')['detailed_view'].agg(list).reset_index()
    
    # Оставляем только пользователей с более чем одним просмотром
    user_history = user_history[user_history['detailed_view'].map(len) > 1]
    
    # Сбрасываем индекс
    user_history = user_history.reset_index(drop=True)
    
    return user_history

# Использование:
detailed_history_df = create_detailed_user_history(result_with_video_info)

# Посмотреть результат
detailed_history_df.head()

,viewer_uid,detailed_view
0,10000001,"[Новая Битва экстрасенсов, 24 сезон, 10 выпуск..."
1,10000002,[Сериал Теория большого взрыва | The Big Bang ...
2,10000003,[Оппенгеймер | Oppenheimer (2023) Путешествия ...
3,10000004,"[Тихоходная братва, лунный самурай и пересадка..."
4,10000007,[Почему предателей не любят нигде? — Документа...


In [118]:
def find_nan_in_history(df):
    # Проверяем наличие NaN в каждом списке detailed_view
    nan_rows = df[df['detailed_view'].apply(lambda x: any(pd.isna(i) for i in x))]
    
    return nan_rows

# Использование:
nan_rows_df = find_nan_in_history(detailed_history_df)

# Посмотреть результат
print(nan_rows_df)

Empty DataFrame
Columns: [viewer_uid, detailed_view]
Index: []


In [119]:
# Убираем префикс 'video_' из rutube_video_id
# result_with_video_info['clean_video_id'] = result_with_video_info['rutube_video_id'].str.replace('video_', '')

# # Фильтруем строки, где clean_video_id равно '331710'
# filtered_row = result_with_video_info[result_with_video_info['clean_video_id'] == '368413']

In [120]:
# filtered_row[filtered_row['viewer_uid'] == 10000026]

In [121]:
detailed_history_df

,viewer_uid,detailed_view
0,10000001,"[Новая Битва экстрасенсов, 24 сезон, 10 выпуск..."
1,10000002,[Сериал Теория большого взрыва | The Big Bang ...
2,10000003,[Оппенгеймер | Oppenheimer (2023) Путешествия ...
3,10000004,"[Тихоходная братва, лунный самурай и пересадка..."
4,10000007,[Почему предателей не любят нигде? — Документа...
...,...,...
907978,11140868,"[Маг на полную ставку, 2 сезон, 10 серия Аниме..."
907979,11140872,[КТО ПОСЛЕДНИЙ ЛОПНЕТ ГРЕЛКУ / Стальные легкие...
907980,11140878,[НОВЫЕ ДОРАМЫ ♦ 10 классных СЕРИАЛОВ ♦ Корейск...
907981,11140885,[Гравити Фолз сезон 1 серия 12 «Летоуин» (муль...


In [112]:
user_history_df['clean_video_id'][13]

['368413',
 '199861',
 '219305',
 '251129',
 '396926',
 '387773',
 '371881',
 '340135',
 '34440',
 '94312',
 '114681',
 '84617',
 '18465',
 '310865',
 '34440',
 '118475',
 '373032',
 '282569',
 '458664',
 '427386',
 '117988',
 '436377',
 '306356',
 '22391',
 '293503',
 '254791',
 '267306',
 '356246',
 '257938',
 '316595',
 '394229',
 '384879',
 '23169',
 '149393',
 '328183',
 '341877',
 '209045',
 '245952',
 '46178',
 '416502',
 '476766',
 '472203',
 '332551',
 '274936',
 '346907',
 '363003',
 '473428',
 '27996',
 '230363',
 '308266',
 '229439']

In [122]:
detailed_history_df['detailed_view'][13]

['[Ч.1] Почему после 25 лет в США она внезапно решила переехать обратно в Россию #Иммиграция @sfilinom Интервью просмотрено в приложении 01 июня',
 'Война и мир / Картину мира нам диктуют два крупных издателя Интервью просмотрено в приложении 01 июня',
 'Решение скандальное. Мы не можем оставаться в стороне и просто это не замечать Интервью просмотрено в приложении 01 июня',
 'Дмитрий Goblin Пучков: Шойгу, Белоусов, Пригожин /// ЭМПАТИЯ МАНУЧИ Интервью просмотрено в приложении 03 июня',
 '[Ч.2] Толерантность и Деградация. Почему после 25 лет в США она едет в Россию #иммиграция @sfilinom Интервью просмотрено в приложении 05 июня',
 'Серьёзный разговор с Главой Республики Крым — Сергеем Аксёновым Интервью просмотрено в приложении 07 июня',
 'Dj Polkovnik - САМЫЕ ЛУЧШИЕ ТРЕКИ 2023🔥 Часовой предновогодний топ-микс. Techno, Trance, House, EDM Музыка просмотрено в приложении 07 июня',
 'Скайнет Йоханссон, проблемная крыша Google, эволюция роботов | В цепких лапах Технологии и интернет просмо

In [96]:
filtered_row

,event_timestamp,ua_device_type,ua_client_type,ua_os,ua_client_name,rutube_video_id,viewer_uid,title,category,duration,author_id,formatted_date,client_type,detailed_view,clean_video_id
5987,2024-06-01 15:38:33+03:00,smartphone,mobile app,Android,Rutube,video_331710,10564819,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,01 июня,приложении,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710
18613,2024-06-01 18:14:43+03:00,desktop,browser,Windows,Chrome,video_331710,10022916,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,01 июня,браузере,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710
103715,2024-06-02 18:55:20+03:00,tablet,mobile app,Android,Rutube,video_331710,10764566,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,02 июня,приложении,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710
160693,2024-06-03 12:58:35+03:00,desktop,browser,Windows,Chrome,video_331710,10224522,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,03 июня,браузере,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710
162299,2024-06-03 06:45:29+03:00,desktop,browser,Windows,Chrome,video_331710,10008083,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,03 июня,браузере,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10130753,2024-06-30 01:49:00+03:00,smartphone,mobile app,Android,Rutube,video_331710,10635116,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,30 июня,приложении,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710
10143562,2024-06-30 00:11:21+03:00,smartphone,mobile app,Android,Rutube,video_331710,10245606,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,30 июня,приложении,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710
10163009,2024-06-30 12:20:36+03:00,smartphone,mobile app,Android,Rutube,video_331710,10838767,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,30 июня,приложении,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710
10165383,2024-06-30 05:10:59+03:00,desktop,browser,Windows,Yandex Browser,video_331710,10881537,Плохие парни | Bad Boys (1995),Фильмы,7131136,1056177,30 июня,браузере,Плохие парни | Bad Boys (1995) + Фильмы + прос...,331710


In [98]:
user_history_df['clean_video_id'][73]

['331710',
 '404061',
 '414732',
 '212467',
 '17138',
 '37931',
 '89575',
 '13372',
 '316595',
 '341877']

In [138]:
def create_user_description(targets_df):
    # Находим максимальное время просмотра для каждого пользователя
    max_watch_time = targets_df.groupby('viewer_uid')['total_watchtime'].transform('max')
    
    # Фильтруем строки с максимальным временем просмотра
    filtered_df = targets_df[targets_df['total_watchtime'] == max_watch_time]
    
    # Создаем описание для каждого пользователя
    def create_description(group):
        row = group.iloc[0]  # берем первую строку для базовой информации
        
        # Определяем пол
        gender = "мужчина" if row['sex'] == 'male' else "женщина"
        
        # Собираем все уникальные регионы
        regions = ' и '.join(group['region'].unique())
        
        # Формируем описание
        description = (
            f"{gender}, {row['age']} лет, "
            f"живет в {regions}, "
            f"смотрит на сайте {row['total_watchtime']} секунд"
        )
        
        return description
    
    # Группируем по viewer_uid и создаем описания
    user_descriptions = (filtered_df.groupby('viewer_uid')
                        .apply(create_description)
                        .reset_index()
                        .rename(columns={0: 'user_description'}))
    
    return user_descriptions
user_descriptions = create_user_description(targets)
user_descriptions.head()

,viewer_uid,user_description
0,10000001,"женщина, 40 лет, живет в Moscow, смотрит на са..."
1,10000002,"мужчина, 44 лет, живет в Moscow, смотрит на са..."
2,10000004,"мужчина, 36 лет, живет в Moscow и Vologda Obla..."
3,10000005,"мужчина, 38 лет, живет в Tatarstan Republic, с..."
4,10000006,"мужчина, 38 лет, живет в Sverdlovsk Oblast, см..."


In [143]:
user_descriptions['user_description'][843]

'мужчина, 51 лет, живет в St.-Petersburg и Tver Oblast, смотрит на сайте 2520 секунд'

In [126]:
detailed_history_df

,viewer_uid,detailed_view
0,10000001,"[Новая Битва экстрасенсов, 24 сезон, 10 выпуск..."
1,10000002,[Сериал Теория большого взрыва | The Big Bang ...
2,10000003,[Оппенгеймер | Oppenheimer (2023) Путешествия ...
3,10000004,"[Тихоходная братва, лунный самурай и пересадка..."
4,10000007,[Почему предателей не любят нигде? — Документа...
...,...,...
907978,11140868,"[Маг на полную ставку, 2 сезон, 10 серия Аниме..."
907979,11140872,[КТО ПОСЛЕДНИЙ ЛОПНЕТ ГРЕЛКУ / Стальные легкие...
907980,11140878,[НОВЫЕ ДОРАМЫ ♦ 10 классных СЕРИАЛОВ ♦ Корейск...
907981,11140885,[Гравити Фолз сезон 1 серия 12 «Летоуин» (муль...


In [144]:
detailed_history_df.to_parquet('textual_history.parquet')
user_history_df.to_parquet('id_history.parquet')
user_descriptions.to_parquet('user_descriptions.parquet')